# Get stats on ions based on mz delta to khipu primary peaks

For 45 Orbitrap datasets from the CSM project. Use delta to khipu primary peaks, no need for full pairwise (see Step 1).

We take histogram counts per dataset, as KDE is not easy to automate on less dense data of varying quality. Peak detection is done on cumulative histogram.

This produces most frequent mass delta values for pos and neg ionization on Orbitraps.

SL 2024-11-19

## Imports

In [91]:
import io
import sys
import tqdm
import contextlib
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import find_peaks 
from scipy.ndimage import uniform_filter1d
import statsmodels.api as sm
from khipu.extended import peaklist_to_khipu_list, export_empCpd_khipu_list
import importlib
sys.path.insert(0, '..')
from mining import * 
from isf_helper import calculate_bin_deltas_fulldataset

## Loads necessary information

In [ ]:
import requests, zipfile, os
download_path = '../input_data_orbi.zip'
extract_path = '..'

# download from zenodo take 20sec-2min. Only need to do once. 
try:
    with requests.get('https://zenodo.org/records/14541717/files/isf_input_data_orbi.zip?download=1', stream=True) as response:
        response.raise_for_status()

        with open(download_path, 'wb') as file:
                for chunk in response.iter_content(chunk_size=8192):
                    file.write(chunk)
    print(f"Downloaded file to: {os.path.abspath(download_path)}")

    with zipfile.ZipFile(download_path, 'r') as z:
            z.extractall(path=extract_path)
            print(f"Extracted files to: {os.path.abspath(extract_path)}")

except requests.exceptions.RequestException as e:
    print(f"Error during download: {e}")
except zipfile.BadZipFile:
    print("The file is not a valid zip file.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")
finally:
    if os.path.exists(download_path):
        os.remove(download_path)
        print(f"Deleted downloaded file: {os.path.abspath(download_path)}")

In [92]:
orbi_datasets = [x.rstrip() for x in open('selected_45_orbi_datasets.txt').readlines()]

In [93]:
def get_isotope_elution_window(good_khipus):
    '''good_khipus must have M0 as a good peak (snr>=5, shape>=0.9)
    
    good_khipus: a list of khipus

    return: Mean and stdev of M0 peak width; mean and stdev of (M1-M0 retention time shift)
    '''
    M0wdiths, M1shifts = [], []
    for epd in good_khipus:
        M0, M1 = get_M0(epd['MS1_pseudo_Spectra']), get_M1(epd['MS1_pseudo_Spectra'])
        M0wdiths.append(M0['right_base'] - M0['left_base'])
        M1shifts.append(M1['rtime'] - M0['rtime'])
        
    mean_m0, stdev_m0 = np.mean(M0wdiths), np.std(M0wdiths)
    mean_shift, stdev_shift = np.mean(M1shifts), np.std(M1shifts)

    return mean_m0, stdev_m0, mean_shift, stdev_shift

def summarize_dataset(f):
    '''called different functions to generate necessary information for following calculation
    
    f: the name of dataset. Ex, ST001237_HILICpos_B2_ppm5_3524314
    
    return: stdev of M1-M0 retention time shift, full list of features and list of good khipus
    '''
    
    def path_constructor(f):
        '''constructor to generate ion mode and paths for input files
        
        f: the name of dataset. Ex, ST001237_HILICpos_B2_ppm5_3524314
        
        return: ion mode, full feature table and preferred feature table
        '''
        # specific for this analysis
        ion_mode = 'pos' if 'pos' in f else 'neg'
        full_feature_table = f'../input_data_orbi/{f}/full_feature_table.tsv'
        preferred_feature_table = f'../input_data_orbi/{f}/preferred_feature_table.tsv'
        
        return ion_mode, full_feature_table, preferred_feature_table

    def generate_ecpds_from_feature_path(path, ion_mode):
        '''generate empirical compounds json from the path of full feature table
        
        path: the path to full feature table
        ion_mode: 'pos' or 'neg'
        
        returns: the path to the generated ecpds.json. Will be under input_data_orbi/{f}/. f is the current dataset name.
        '''
        KCD_formula_coordinate = build_KCD_from_formula_coordinate(formula_coordinate)
        num_samples, st = read_features_from_asari_table(open(path).read())
        mass_accuracy_ratio, st = custom_mz_calibrate(st, KCD_formula_coordinate, 
															mode=ion_mode, 
															mz_tolerance_ppm=5, 
															required_calibrate_threshold=0.000002)
        _json_empcpd = os.path.join(f'../input_data_orbi/{f}/ecpds.json')
        features2cpds(st, mode=ion_mode, mz_tolerance_ppm=5, rt_tolerance=2, outfile=_json_empcpd)
        return _json_empcpd
    
    with contextlib.redirect_stdout(io.StringIO()):
        ion_mode, full_Feature_table, _ = path_constructor(f)
        if not os.path.exists(f'../input_data_orbi/{f}/ecpds.json'):
            ecpds_path = generate_ecpds_from_feature_path(full_Feature_table, ion_mode)
        else:
            ecpds_path = f'../input_data_orbi/{f}/ecpds.json'
        _, _, epd_summary = epd2featurelist_from_file(ecpds_path, mode=ion_mode)
        _, _, _, stdev_shift = get_isotope_elution_window(epd_summary['good_khipus'])
        _, featureList = read_features_from_asari_table(open(full_Feature_table).read())
    return stdev_shift, featureList, epd_summary['good_khipus']
    

## Calculate Mass Deltas Across Datasets

In [94]:
# ~1h for first time. Once ecpds generated, <20min
collection_deltas = [] # the list of mz delta value list of each dataset
for orbi_dataset in tqdm.tqdm(orbi_datasets):
    stdev_shift, st, good_khipus = summarize_dataset(orbi_dataset)
    mz_delta_list = calculate_bin_deltas_fulldataset(good_khipus, stdev_shift, st)
    collection_deltas.append(mz_delta_list)  

100%|██████████| 45/45 [16:21<00:00, 21.81s/it]


# Explained features by intensity quartiles

Get khipus using updated grid,
Add new list of fragments
- Collect features that are explained;
- and numbers of unique empCpds

Plot how they overlap by each incremental quartile of features. (Done in step5)

The candidate fragments are based on frequent delta m/zs. 

Not using those from MS2 as the freq lists here should be more inclusive.

Will move the frequent lists and top ones into mass2chem and khipu packages.

## Generate Elution Parameters for Next Steps

In [96]:
elution_parameters = []
for f in orbi_datasets:
    _lf, f2epd, epd_summary = epd2featurelist_from_file(f'../input_data_orbi/{f}/ecpds.json',  mode='pos' if 'pos' in f else 'neg')
    elution_parameters.append((f, epd_summary['num_good_khipus'], get_isotope_elution_window(epd_summary['good_khipus'])))
    
s = 'ID\tnumber_good_khipus\tmean_m0\tstdev_m0\tmean_shift\tstdev_shift\n'    
# flatten the list
for ep in elution_parameters:
    s += '\t'.join([str(round(item, 3) if isinstance(item, float) else item) for sublist in ep for item in (sublist if isinstance(sublist, tuple) else (sublist,))]) + '\n'    
        
with open('elution_parameters_45studies_orbi.tsv', 'w') as O:
    O.write(s)

## Calculate Histogram and KDE for Each Dataset
The result will be under freq_mzdelta

In [97]:
def get_histogram_per_dataset(list_kp_deltas, binsize=0.001):
    list_deltas = []
    for v in list_kp_deltas:
        list_deltas += v[1]
    list_deltas = np.array(list_deltas)
    _min, _max = list_deltas.min(), list_deltas.max()
    bins = np.arange(_min, _max, binsize)
    hist = np.histogram(list_deltas, bins)
    return hist

def write_top_histo_bins(f, list_kp_deltas, binsize=0.001, outdir='../freq_mzdelta_orbi/', topN=500):
    os.makedirs(outdir, exist_ok=True)
    
    h1, b2 = get_histogram_per_dataset(list_kp_deltas, binsize=binsize)
    new = list(zip(h1, b2))
    new.sort(reverse=True)
    s = 'count\tbin_left_edge\n'
    for x in new[:topN]:
        s += str(x[0]) + '\t' + str(round(x[1], 4)) + '\n'
    with open(outdir+'histo_'+f+'.tsv', 'w') as O:
        O.write(s)

def write_top_kde_peaks(f, peaks_density, outdir='../freq_mzdelta_orbi/'):
    os.makedirs(outdir, exist_ok=True)
    
    # peaks_density : [(mz, density), ...]
    s = 'mz_peak\tKDE_density\n'
    for x in peaks_density:
        s += str(round(x[0], 4)) + '\t' + str(round(x[1], 4)) + '\n'
    with open(outdir+'kde_'+f+'.tsv', 'w') as O:
        O.write(s)

def get_peaks(x_hist, y_hist, 
                  height=0.01,
                  distance=2,
                  # prominence=0.5,
                  width=2,
                  wlen=50,
                  ):
    # prominence is not used as height is sufficient after smoothing first
    y_hist_smoothed = uniform_filter1d(y_hist, 5, mode='nearest')
    peaks, properties = find_peaks(y_hist_smoothed, 
                                    height=height, 
                                    distance=distance,
                                    # prominence=prominence,
                                    width=width, 
                                    wlen=wlen,
                                    ) 
    real_apexes = [x_hist[ii] for ii in peaks]
    return list(zip(real_apexes, properties['peak_heights']))

def get_kde_per_dataset(list_kp_deltas, bandwidth=0.001, threshold=0.001, topN=500):
    '''
    Testing. Not using in final results.
    '''
    list_deltas = []
    for v in list_kp_deltas:
        list_deltas += v[1]
    kde = sm.nonparametric.KDEUnivariate(list_deltas)
    kde.fit(bw=bandwidth) 
    max_intensity = kde.density.max()
    threshold = min(threshold, threshold*max_intensity)
    # prominence = 0.05 * max_intensity
    peaks_density = get_peaks(kde.support, kde.density, height=threshold, 
                                        # prominence=prominence,
                                        )
    peaks_density = sorted(peaks_density, key=lambda x: x[1], reverse=True)
    return peaks_density[:topN]

In [98]:
for orbi_dataset, deltas_per_dataset in zip(orbi_datasets, collection_deltas): 
    # do histogram
    write_top_histo_bins(orbi_dataset, deltas_per_dataset)    
    
    # do KDE
    peaks_density = get_kde_per_dataset(deltas_per_dataset, bandwidth=0.0005, threshold=0.001, topN=500)
    write_top_kde_peaks(orbi_dataset, peaks_density)

**Note**

Tricky to optimize KDE peak finding parameters for noisier data. When overall density is low, peak shapes are not clear.

Histogram is robust to use here.

## Get Cumulative Histogram and KDE across Datasets

In [99]:
def get_histogram_fixed_bins(list_kp_deltas, fixed_bins):
    '''Returns histogram on fixed bins using abs values
    
    list_kp_deltas: list of tuples containing khipu id and related list of delta mz values. 
        Ex, [('kp4_69.0587', [0.0,
                5.033599999999993,
                22.167199999999994,
                11.455299999999994,
                1.0032999999999959]),
            ('kp5_69.0587', [0.0,
                10.871099999999998,
                1.0032999999999959])]
    fixed_bins: an 1D numpy array of evenly spaced floating-point numbers. Ex. np.arange(0.1, 310, 0.0001)
    
    return: 1D numpy array representing counts of all deltas in current dataset in each bin given by fixed_bins. Ex. array([22.,  6., 14., ...,  7.,  5.,  4.], shape=(3098999,))
    '''
    list_deltas = []
    for v in list_kp_deltas:
        list_deltas += v[1]
    np_list_deltas = abs(np.array(list_deltas))
    return np.histogram(np_list_deltas, fixed_bins)[0]

In [100]:
fixed_bins = np.arange(0.1, 310, 0.0001)
cummulative_histo = np.zeros(len(fixed_bins)-1)

for d in collection_deltas:
    cummulative_histo += get_histogram_fixed_bins(d, fixed_bins)
    
cummulative_kde = get_peaks(fixed_bins[:-1], cummulative_histo, height=100)
cummulative_kde = sorted(cummulative_kde, key=lambda x: x[1], reverse=True)

## Curate Mass Signatures

In [102]:
from mass2chem.lib.common_mass import mass_signatures
# remove F
mass_signatures = [ms for ms in mass_signatures if 'F' not in ms[2]]

def search_mass_signatures(m, mass_signatures=mass_signatures):
    '''find the closest mass signatures for the input mass value
    
    m: float. The input mass value
    mass_signatures: list of tuples containing mass value, chem formula, single element dict of each mass signature. 
    
    return: the closest signature and the deviation.
        
    example:
    >>> search_mass_signatures(28.0312)
    ((28.0313, '± C2H4, natural alkane chains such as fatty acids', {'C': 2, 'H': 4}), 0.00010000000000331966)
    '''
    _d = [abs(m-abs(x[0])) for x in mass_signatures]
    # find all mass signatures whose notes are not empty strings.
    _iis = [i for i, delta in enumerate(_d) if delta == min(_d)]
    if _d[_iis[0]] < 0.0005:
        if len(_iis) > 1:
            if filtered_iis := [_i for _i in _iis if (mass_signatures[_i][1] != '' and mass_signatures[_i][0] > 0)]:
                _ii = filtered_iis[0]
            else:
                _ii = _iis[0]
        else:
            _ii = _iis[0]
        return mass_signatures[_ii], _d[_ii]
    else:
        return None
    
def write_delta_mzs(output_path, mz_delta_kdes, mass_signatures):
    '''write the top frequent delta mzs into designated tsv file
    
    output_path: the path of top frequent delta mzs
    mz_delta_kdes: list of tuples containing mz delta values and kdes. 
                    Ex,[(1.0033000000000258, 8393.8),
                        (14.015500000000399, 1590.6),
                        (2.015500000000055, 1285.8),
                        (18.010500000000516, 1244.8)]
    mass_signatures: list of tuples containing mass value, chem formula, single element dict of each mass signature. 
    '''
    s = 'delta_mz\tcount_estimate\tmass_signature\tnote\tdict\n'
    for x in mz_delta_kdes:
        _M = search_mass_signatures(x[0], mass_signatures)
        if _M:
            s += str(round(x[0], 4)) + '\t' + str(int(x[1])) + '\t' + '\t'.join(
                [str(ii) for ii in _M[0] ]) + '\n'
        else:
            s += str(round(x[0], 4)) + '\t' + str(int(x[1])) + '\n'
            
    with open(output_path, 'w') as O:
        O.write(s)

In [103]:
curated_msig = []
for x in cummulative_kde:
    _M = search_mass_signatures(x[0])
    if _M:
        curated_msig.append((float(x[0]), _M))
        
write_delta_mzs('top_frequent_delta_mz_orbi_anno-20250128.tsv', cummulative_kde, curated_msig)

In [104]:
len(curated_msig)

279

## Do pos and neg ions separately

In [105]:
import sys
# Remove the module from sys.modules
sys.modules.pop('mass2chem.lib.common_mass', None)
# Re-import the module
from mass2chem.lib.common_mass import mass_signatures

In [106]:
fixed_bins = np.arange(0.1, 500.01, 0.0001)
cummulative_histo_pos = np.zeros(len(fixed_bins)-1)
cummulative_histo_neg = np.zeros(len(fixed_bins)-1)

for orbi_dataset, deltas_per_dataset in zip(orbi_datasets, collection_deltas): 
    if 'pos' in orbi_dataset:
        cummulative_histo_pos += get_histogram_fixed_bins(deltas_per_dataset, fixed_bins)
    elif 'neg' in orbi_dataset:
        cummulative_histo_neg += get_histogram_fixed_bins(deltas_per_dataset, fixed_bins)
    else:
        print("Error, ", orbi_dataset)

In [107]:
# pos
peaks_density_pos = get_peaks(fixed_bins[:-1], cummulative_histo_pos, height=100)
peaks_density_pos = sorted(peaks_density_pos, key=lambda x: x[1], reverse=True)

write_delta_mzs('top_frequent_delta_mz_orbi_pos-20250128.tsv', peaks_density_pos, mass_signatures)

In [108]:
# neg
peaks_density_neg = get_peaks(fixed_bins[:-1], cummulative_histo_neg, height=100)
peaks_density_neg = sorted(peaks_density_neg, key=lambda x: x[1], reverse=True)

write_delta_mzs('top_frequent_delta_mz_orbi_neg-20250128.tsv', peaks_density_neg, mass_signatures)

## Conclusion

We have calculated most frequent mz deltas in Orbitrap datasets.

Rough annotation is drawn from mass2chem etc. 

Will reformat the tables and update with full annotation.